In [8]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# Download Penn Treebank dataset
import urllib.request
import os

# Create data directory if it doesn't exist
os.makedirs('data/ptb', exist_ok=True)

# Penn Treebank URLs
ptb_urls = {
    'train': 'https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb.train.txt',
    'valid': 'https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb.valid.txt',
    'test': 'https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb.test.txt'
}

# Download files
for split, url in ptb_urls.items():
    filepath = f'data/ptb/ptb.{split}.txt'
    if not os.path.exists(filepath):
        print(f'Downloading {split} data...')
        urllib.request.urlretrieve(url, filepath)
        print(f'Saved to {filepath}')
    else:
        print(f'{split} data already exists at {filepath}')

Saved to data/ptb/ptb.train.txt
Saved to data/ptb/ptb.valid.txt
Saved to data/ptb/ptb.test.txt
Saved to data/ptb/ptb.valid.txt
Saved to data/ptb/ptb.test.txt


In [1]:
# Load and tokenize the data
def load_ptb_data(filepath):
    """Load PTB data and return list of words"""
    with open(filepath, 'r') as f:
        # PTB has one sentence per line, words separated by spaces
        # <unk> is used for unknown words, <eos> marks end of sentence
        text = f.read()
        words = text.split()
    return words

# Load all splits
train_words = load_ptb_data('data/ptb/ptb.train.txt')
valid_words = load_ptb_data('data/ptb/ptb.valid.txt')
test_words = load_ptb_data('data/ptb/ptb.test.txt')

print(f'Train: {len(train_words)} words')
print(f'Valid: {len(valid_words)} words')
print(f'Test: {len(test_words)} words')
print(f'\nFirst 50 words: {train_words[:50]}')

Train: 887521 words
Valid: 70390 words
Test: 78669 words

First 50 words: ['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', 'pierre', '<unk>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', 'mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group']


In [18]:
def build_vocabulary(words):
    words_set = set(words)
    word_to_idx = {word: i for i, word in enumerate(words_set)}
    idx_to_word = {i: word for (word, i) in word_to_idx.items()}

    return word_to_idx, idx_to_word

In [19]:
word_to_idx, idx_to_word = build_vocabulary(train_words+valid_words+test_words)
Vsize = len(word_to_idx)

In [26]:
Vsize

9999

In [50]:
def build_dataset(words):
    X, Y = [], []
    for i in range(len(words)-5):
        x1, x2, x3, x4, y = words[i+1],words[i+2], words[i+3], words[i+4], words[i+5]
        X.append((word_to_idx[x1],word_to_idx[x2],word_to_idx[x3], word_to_idx[x4]))
        Y.append(word_to_idx[y])
    
    return torch.tensor(X),torch.tensor(Y)


In [51]:
trainX, trainY = build_dataset(train_words)
devX, devY = build_dataset(valid_words)
testX, testY = build_dataset(test_words)

In [43]:
trainX.shape

torch.Size([887516, 4])

In [ ]:
batch_size = (256,)
base_lr = 0.5
weight_decay = 0.00001 
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((Vsize, 10), generator=g) * 0.01
W1 = torch.randn((40, 100), generator=g) * 0.01
b1 = torch.zeros((100,))
W2 = torch.randn((100, Vsize), generator=g) * 0.01
b2 = torch.zeros((Vsize,)) 
parameters = [C, W1, b1, W2, b2] 
for p in parameters:
    p.requires_grad = True

In [ ]:
def compute_loss(X, Y):
    with torch.no_grad():
        batch_size = 10_000
        losses = []
        for i in range(0, len(X), batch_size):
            emb = C[X[i:i+batch_size]] 
            emb = emb.view(emb.shape[0], -1) # (887517, 30)
            h = torch.tanh(emb @ W1 + b1) # (887517, 100)
            logits = h @ W2 + b2
            loss = F.cross_entropy(logits, Y[i:i+batch_size]) # Softmax & cross entropy loss 
            losses.append(loss.item())
        return sum(losses)/len(losses)

In [58]:
import torch.nn.functional as F


for i in range(200_000):
    #Forward pass
    ix = torch.randint(trainX.shape[0], batch_size, generator=g)
    emb = C[trainX[ix]] # (887516, 4, 10)
    emb = emb.view(emb.shape[0], -1) # (887516, 40)
    h = torch.tanh(emb @ W1 + b1) # (887516, 100)
    logits = h @ W2 + b2 # (Vsize, )
    loss = F.cross_entropy(logits, trainY[ix]) # Softmax & cross entropy loss compute
    total_loss = loss + weight_decay * sum((p**2).sum() for p in [C, W1, W2])

    if not i % 10_000:
        print(f"step: {i:7d}, mini-batch loss = {loss.item():.4f}")
    
    for p in parameters:
        p.grad = None
    
    # Compute gradients
    total_loss.backward()
    
    # Update parameters
    lr = base_lr if i < 150_000 else base_lr * 0.1
    for p in parameters:
        p.data -= lr * p.grad

print(f"Final training loss: {compute_loss(trainX, trainY)}")
print(f"Final dev loss: {compute_loss(devX, devY)}")

    
    

step:       0, mini-batch loss = 9.2102


step:   10000, mini-batch loss = 6.5817
step:   20000, mini-batch loss = 6.3500
step:   30000, mini-batch loss = 6.9078
step:   40000, mini-batch loss = 6.4351
step:   50000, mini-batch loss = 6.7682
step:   60000, mini-batch loss = 6.2800
step:   70000, mini-batch loss = 6.2917
step:   80000, mini-batch loss = 5.9110
step:   90000, mini-batch loss = 6.4875
step:  100000, mini-batch loss = 6.0302
step:  110000, mini-batch loss = 6.3219
step:  120000, mini-batch loss = 6.1740


KeyboardInterrupt: 

In [ ]:
# Sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

with torch.no_grad():
    for _ in range(10):  # Generate 10 sequences
        out = []
        # Start with 3 random words from vocabulary
        context = torch.randint(0, Vsize, (4,), generator=g).tolist()
        
        for _ in range(20):  # Generate up to 50 words
            # Forward pass
            emb = C[torch.tensor([context])]  # (1, 4, 10)
            h = torch.tanh(emb.view(1, -1) @ W1 + b1)  # (1, 100)
            logits = h @ W2 + b2  # (1, Vsize)
            probs = F.softmax(logits, dim=1) #(1, Vsize)
            
            # Sample next word
            ix = torch.multinomial(probs, num_samples=1, generator=g).item()
            out.append(idx_to_word[ix])
            
            # Update context window (sliding window)
            context = context[1:] + [ix]
        
        print(' '.join(out))

news commission vetoed but a truly image following a $ N million of <unk> <unk> a homeless and weyerhaeuser co.
is thought to be excellent at the split 's <unk> and electronics instruments call but is their jitters on the
displays warner <unk> president of the delegation of crown technical contribution was sensitive to listen to beverage furs on u.s.
or high futures is considering such <unk> state as a response to coats airline this british depositary shares were in
<unk> of goods from unidentified items he said her remarks burned by international express tom <unk> president and a two
at other property dealers had to engineers the <unk> light if people do n't know how the boys quit of
companies which moved on the market despite long although potential costs to six months in october was a book value
in the national association and nynex corp. account manager mr. pilson many of the scientists ' crusaders and recent <unk>
is in school charles mullins a <unk> after he believes the great ju